## Filter
One of the most powerful parameters when defining a search is the ability to filter the results.  While a Query is more of an unstructured approach to search for text within documents, a Filter provides a well-defined structured approach when we know which properties we want to restrict our matching documents.

The following examples will be using some of the popular properties as outlined within the "Properties & Metadata" notebook.  In addition, the selection of these properties was determined based on the discovery techniques used such as metadata and debugging.

In [6]:
import refinitiv.data as rd
from refinitiv.data.content import search
import pandas as pd
import datetime as dt
import dateutil.relativedelta

# Default session - desktop
rd.open_session(app_key='Your API Key here')

<refinitiv.data.session.Definition object at 0x1f615aa2070 {name='default'}>

In [7]:
pd.set_option('display.max_colwidth', 140)
rd.__version__

'1.0.0b9'

#### Example - Query vs Filter
As you become more familiar with Search, you will realize there are a number of different paths you can take to arrive at the same destination.

In [8]:
# Retrieve the document(s) for a specific ISIN using a Query.
response=search.Definition(
    view = search.SearchViews.INSTRUMENTS,
    query = 'US046353AQ14',
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
).get_data()
response.data.df

,DocumentTitle,RIC,ISIN,CUSIP,SEDOL,PermID
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",046353AQ1=,US046353AQ14,046353AQ1,BF4SNM7,46645702869


In [9]:
# Retrieve the document(s) for the same ISIN using a Filter
response=search.Definition(
    view = search.SearchViews.INSTRUMENTS,
    filter = "ISIN eq 'US046353AQ14'",
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
).get_data()
response.data.df

,DocumentTitle,RIC,ISIN,CUSIP,SEDOL,PermID
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",046353AQ1=,US046353AQ14,046353AQ1,BF4SNM7,46645702869


In [10]:
# If you are provided a code but are not sure if the code is an ISIN, SEDOL, CUSIP etc, 
# try using the 'CodeResolutionBase' property.
response=search.Definition(
    view = search.SearchViews.INSTRUMENTS,
    filter = "CodeResolutionBase eq '046353AQ1'",   # I'm using the CUSIP for the same instrument
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
).get_data()
response.data.df

,DocumentTitle,RIC,ISIN,CUSIP,SEDOL,PermID
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",046353AQ1=,US046353AQ14,046353AQ1,BF4SNM7,46645702869


#### Example - Apple Equities

In [11]:
# Within the view (EquityQuotes), select all documents where the term 'AAPL' is within the ticker symbol.  For each, 
# ensure the type of asset is an 'equity' and that each asset is in an active state.  I chose to filter out the 
# AssetState of 'DC' (De-activated) as my criteria. By default, the top 10 results are presented.
#
# Note: An alternative to "AssetType eq 'equity'" could be to use: "RCSAssetCategoryLeaf eq 'Ordinary Share'"
response=search.Definition(
    view = search.SearchViews.EQUITY_QUOTES,
    filter = "AssetType eq 'equity' and AssetState ne 'DC' and TickerSymbol eq 'AAPL'",
    select = "RIC, CompositeDescriptiveName, ExchangeCode, ExchangeName, AssetType"
).get_data()
response.data.df

,RIC,CompositeDescriptiveName,ExchangeCode,ExchangeName,AssetType
0,AAPL.O,Apple Ord Shs APPLE ORD,NSQ,NASDAQ Global Select Consolidated,[EQUITY]
1,AAPLEUR.xbo,Apple Ord Shs APPLE ORD APPLE INC,XBO,MiFID Composite Cross Market Service,[EQUITY]
2,AAPL.OQ,Apple Ord Shs APPLE ORD,NSM,NASDAQ Stock Exchange Global Select Market,[EQUITY]
3,AAPL.Z,Apple Ord Shs APPLE ORD APPLE INC,GCZ,Cboe BZX Exchange - Nasdaq Global Select Market,[EQUITY]
4,AAPL.DG,Apple Ord Shs APPLE ORD,GCD,Cboe EDGX Exchange - Nasdaq Global Select Market,[EQUITY]
5,AAPL.B,Apple Ord Shs APPLE ORD,GSB,Boston SE when trading NASDAQ Global Select Market,[EQUITY]
6,AAPL.CCP,Apple 100 CDR Representing 12 Ord Shs APPLE 100 CDR REP 12 ORD,CAQ,Canadian Composite Quote/Trade,[EQUITY]
7,AAPLE.MI,Apple Ord Shs APPLE ORD,MIL,Milan Stock Exchange,[EQUITY]
8,AAPL.DY,Apple Ord Shs APPLE ORD,GDA,Cboe EDGA Exchange - Nasdaq Global Select Market,[EQUITY]
9,AAPL.PH,Apple Ord Shs APPLE ORD APPLE INC,GSX,NASDAQ OMX PHLX Global Select Market,[EQUITY]


#### Example - Vessels
List all Very Large Gas Containers (VLGC) that are in service within the last week.

In [12]:
# Build date range expression for the location of the vessel for the last 7 days
date_7_days_ago = dt.datetime.today() - dateutil.relativedelta.relativedelta(days=7)
date_range = f'LocationTimestamp ge {date_7_days_ago.strftime("%Y-%m-%d")} and \
               LocationTimestamp lt {dt.datetime.today().strftime("%Y-%m-%d")}'

In [13]:
# Show the location of each vessel and some general details such as location, destination and coordinates.
# If available, show the top 100 hits.
response=search.Definition(
    view = search.SearchViews.VESSEL_PHYSICAL_ASSETS,
    filter = f"RCSAssetTypeLeaf eq 'VLGC' and VesselStatus eq 'IN SERVICE' and {date_range}",
    select = 'RIC, SynonymSearchAll, DTSource, LocationTimestamp, IMO,Latitude, Longitude, LocationDestination',
    top=100
).get_data()
response.data.df

,RIC,SynonymSearchAll,DTSource,LocationTimestamp,IMO,Latitude,Longitude,LocationDestination
0,C}KY7309491971,Egyptian Liquefied Petroleum Gas Very Large Gas Carrier,Red Sea|Suez Canal Tanker Zone|Egypt,2022-03-21T12:04:24.000Z,9350290,28.726166,33.145763,WADIFEIRAN
1,C}KG7309949881,Japanese Liquefied Petroleum Gas Very Large Gas Carrier,North Pacific Ocean (Japan)|Japan Tanker Zone,2022-03-21T21:13:07.000Z,9902794,25.116718,128.361191,SG SIN [Singapore]
2,C}KY7309461813,Indonesian India Liquefied Petroleum Gas Very Large Gas Carrier,Indonesia|East Indian Ocean|Singapore Tanker Zone,2022-03-19T10:01:02.000Z,8910897,-5.541137,104.602966,TL.SEMANGKA
3,C}KY7309478327,Chinese Indonesian Liquefied Petroleum Gas Very Large Gas Carrier,South China Sea|Manila Tanker Zone,2022-03-21T22:54:28.000Z,9176357,14.464555,120.64579,PH SAR>PH BTN
4,C}KY7309480679,Liquefied Petroleum Gas Very Large Gas Carrier,Andaman Sea|Singapore Tanker Zone|Great Channel Waypoint,2022-03-21T11:07:42.000Z,9208239,6.282205,95.350273,DUBAI [Dubai]
5,C}KY7309492328,Algerian Liquefied Petroleum Gas Very Large Gas Carrier,West Med|West Med Tanker Zone|Algeria,2022-03-18T20:11:55.000Z,9354624,36.052425,-0.258788,BETHIOUA
6,C}KY7309494295,India Liquefied Petroleum Gas Very Large Gas Carrier,East Indian Ocean|Singapore Tanker Zone|Sunda Strait Waypoint,2022-03-21T22:15:04.000Z,9377781,-6.137651,105.676224,IDTSE
7,C}KY7309494990,Chinese Thai Korean Japanese Liquefied Petroleum Gas Very Large Gas Carrier,South China Sea|Thailand Tanker Zone,2022-03-20T13:55:17.000Z,9385685,11.188777,112.538406,SINGAPORE [Singapore]
8,C}KY7309495163,India Liquefied Petroleum Gas Very Large Gas Carrier Thai Vietnamese,East Indian Ocean|Colombo Tanker Zone,2022-03-21T20:36:47.000Z,9387750,5.703752,83.964996,VUNG TAU [Vung Tau]
9,C}KY7309496857,Indian Indonesian India Liquefied Petroleum Gas Very Large Gas Carrier,Arabia Sea|West Coast India Tanker Zone,2022-03-21T19:23:17.000Z,9407122,15.005103,69.154663,SARTA [Ras Tanura]


#### Example - Floating Rate Notes
Retrieve all floating rate notes (floating coupon) in the healthcare sector which are issued in Euros.  For each, ensure the coupon rate is great than 2 and our Moody's rating is greater than 'Baa1'.

To determine the rating, we apply the following logic utilizing the RatingsScope property - which is a nested property providing additional details:

- RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10)

The RatingType of 'MIS' refers to the "Moody's Long-term Issuer Rating".
The CurrentRatingRank < 10 instructs search to look for ratings less than 10 ('Baa1').  **Note**: The lower the rank, the better the rating.

In [14]:
response=search.Definition(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    filter = "CouponRate gt 2 and \
              RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10) and \
              RCSCouponTypeName eq 'Floating Coupon' and \
              IsActive eq true and \
              RCSTRBC2012Name eq 'Healthcare' and \
              RCSCurrencyLeaf xeq 'Euro'",
    select = "CouponCurrency, IssuerName, IssueDate, RIC, CouponRate, ISIN, Ticker, RCSTRBC2012Leaf, MaturityDate, AssetState",   
).get_data()
response.data.df

,CouponCurrency,IssuerName,IssueDate,RIC,CouponRate,ISIN,Ticker,RCSTRBC2012Leaf
0,EUR,Sanofi FR,1983-07-01T00:00:00.000Z,FR0000140022=,15.5,FR0000140022,SASY,Pharmaceuticals (NEC)
1,EUR,Sanofi FR,1987-10-01T00:00:00.000Z,FR0000047656=AURO,15.5,FR0000047656,SASY,Pharmaceuticals (NEC)


#### Example - Outstanding Green Bonds
List the outstanding Green Bonds for Telefonica.

In [15]:
response=search.Definition(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    filter = "IssuerName eq 'Telefonica' and IsGreenBond eq true",
    select = "IssuerName, DocumentTitle, RIC, CouponRate, ISIN, IssuerTicker, DebtTypeDescription, EOMAmountOutstanding, FaceOutstanding"
).get_data()
response.data.df

,IssuerName,DocumentTitle,RIC,CouponRate,ISIN,IssuerTicker,DebtTypeDescription,EOMAmountOutstanding,FaceOutstanding
0,Telefonica Europ,"Telefonica Europe BV, Bond, TEF 2.502",ES210981985=,2.502,XS2109819859,TEFBV,Junior Subordinated Note,500000000,500000000
1,Telefonica Emis,"Telefonica Emisiones SAU, Plain Vanilla Fixed Coupon Bond, TEF 1.069 05-Feb-2024",ES194600445=,1.069,XS1946004451,TEFEMI,Senior Note,1000000000,1000000000


#### Example - Equity Index listing in the U.S.A

In [16]:
# Based on the Asset Category (Equity Index), list the top 10 indexes within the USA.
response=search.Definition(
    view = search.SearchViews.INDEX_INSTRUMENTS,
    filter = "RCSIndexCountryGroupLeaf xeq 'United States' and RCSAssetCategoryLeaf xeq 'Equity Index'",
    select = "DocumentTitle, RIC, RCSTRBC2012Leaf"
).get_data()
response.data.df

,DocumentTitle,RIC,RCSTRBC2012Leaf
0,"S&P 500 Index - CBOE, Equity Index",.SPX,Magazine Publishing
1,"Dow Jones Industrial Average Index, Equity Index",.DJI,Consumer Publishing (NEC)
2,"NASDAQ Composite Index, Equity Index",.IXIC,Financial & Commodity Market Operators & Service Providers (NEC)
3,"NASDAQ 100 Index, Equity Index",.NDX,UK Investment Trusts
4,"Refinitiv United States Price Return Index, Equity Index",.TRXFLDUSP,Professional Information Services (NEC)
5,"Dow Jones Composite Index, Equity Index",.DJA,Consumer Publishing (NEC)
6,"Dow Jones Transport Average Index, Equity Index",.DJT,Consumer Publishing (NEC)
7,"S&P 600 Small Cap Index, Equity Index",.SPCY,Magazine Publishing
8,"Dow Jones Utility Average Index, Equity Index",.DJU,Consumer Publishing (NEC)
9,"NYSE Composite Index, Equity Index",.NYA,Financial & Commodity Market Operators & Service Providers (NEC)


#### Example - Search for Brazil global sovereign bonds in USD with a duration greater than 1

In [17]:
response=search.Definition(
    view=search.SearchViews.FIXED_INCOME_INSTRUMENTS,
    filter = "RCSIssuerCountryLeaf eq 'Brazil' and \
              RCSCurrencyLeaf eq 'US Dollar' and \
              RCSTRBC2012Leaf eq 'Government & Government Finance' and \
              OptionAdjustedDuration gt 1",
    select ="RIC, ISIN, MaturityDate, CouponRate, IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration"
).get_data()
response.data.df

,RIC,ISIN,MaturityDate,CouponRate,IndustrySector,FaceOutstandingUSD,OptionAdjustedDuration
0,BR012631375=,US105756AR10,2024-04-15T00:00:00.000Z,8.875,SOVERGRN,1029030000.0,1.855142
1,BR017244388=,US105756AZ36,2024-04-15T00:00:00.000Z,8.875,SOVERGRN,85557000.0,1.855142
2,105756BV1=,US105756BV13,2025-01-07T00:00:00.000Z,4.25,SOVERGRN,4300000000.0,2.60083
3,BR021210641=,US105756BF62,2025-02-04T00:00:00.000Z,8.75,SOVERGRN,688201000.0,2.562772
4,105756CD0=,US105756CD06,2025-06-06T00:00:00.000Z,2.875,SOVERGRN,1750000000.0,3.005891
5,105756BX7=,US105756BX78,2026-04-07T00:00:00.000Z,6.0,SOVERGRN,2176245000.0,3.514519
6,105756AE0=,US105756AE07,2027-05-15T00:00:00.000Z,10.125,SOVERGRN,765023000.0,4.080794
7,105756BZ2=,US105756BZ27,2028-01-13T00:00:00.000Z,4.625,SOVERGRN,3000000000.0,4.90565
8,105756CA6=,US105756CA66,2029-05-30T00:00:00.000Z,4.5,SOVERGRN,2000000000.0,5.891959
9,BR010862957=,US105756AL40,2030-03-06T00:00:00.000Z,12.25,SOVERGRN,237592000.0,5.732701


In [18]:
# Alternatively, you have the option of trying it this way which will render the same results
# using alternative filter properties, i.e. IssuerCountryName, IndustrySectorDescrioption and Currency
response=search.Definition(
    view=search.SearchViews.FIXED_INCOME_INSTRUMENTS,
    filter = "IssuerCountryName eq 'Brazil' and \
              OptionAdjustedDuration gt 1 and \
              Currency eq 'USD' and IndustrySectorDescription eq 'Sovereign'",
    select ="RIC, ISIN, MaturityDate, CouponRate,IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration"
).get_data()
response.data.df

,RIC,ISIN,MaturityDate,CouponRate,IndustrySector,FaceOutstandingUSD,OptionAdjustedDuration
0,BR012631375=,US105756AR10,2024-04-15T00:00:00.000Z,8.875,SOVERGRN,1029030000.0,1.855142
1,BR017244388=,US105756AZ36,2024-04-15T00:00:00.000Z,8.875,SOVERGRN,85557000.0,1.855142
2,105756BV1=,US105756BV13,2025-01-07T00:00:00.000Z,4.25,SOVERGRN,4300000000.0,2.60083
3,BR021210641=,US105756BF62,2025-02-04T00:00:00.000Z,8.75,SOVERGRN,688201000.0,2.562772
4,105756CD0=,US105756CD06,2025-06-06T00:00:00.000Z,2.875,SOVERGRN,1750000000.0,3.005891
5,105756BX7=,US105756BX78,2026-04-07T00:00:00.000Z,6.0,SOVERGRN,2176245000.0,3.514519
6,105756AE0=,US105756AE07,2027-05-15T00:00:00.000Z,10.125,SOVERGRN,765023000.0,4.080794
7,105756BZ2=,US105756BZ27,2028-01-13T00:00:00.000Z,4.625,SOVERGRN,3000000000.0,4.90565
8,105756CA6=,US105756CA66,2029-05-30T00:00:00.000Z,4.5,SOVERGRN,2000000000.0,5.891959
9,BR010862957=,US105756AL40,2030-03-06T00:00:00.000Z,12.25,SOVERGRN,237592000.0,5.732701


#### Example - Top executives

In [19]:
# Using the People View, we can utilize the OfficerDirector property, which contains a number of nested or sub-properties 
# that allow me to perform a more fine-grained search against active people that have been assigned a title code of 
# CEO (Chief Executive Officer). The results are listed based on a default rank defined by the search service.
response=search.Definition(
    view = search.SearchViews.PEOPLE,
    filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')",
    top = 20,
    select = "FullName, PrimaryTitleAsReported, PrimaryOfficerDirector, RIC, OfficerDirectorActiveCompanies, PrimaryOfficerDirectorRIC"
).get_data()
response.data.df

,FullName,PrimaryTitleAsReported,PrimaryOfficerDirector,OfficerDirectorActiveCompanies,PrimaryOfficerDirectorRIC
0,Satya Nadella,"Chairman of the Board, Chief Executive Officer",Microsoft Corp,"[Starbucks Corp, Microsoft Deutschland GmbH, Chandler Insurance Company Ltd, Microsoft Corp]",MSFT.O
1,Tim D. Cook,"Chief Executive Officer, Director",Apple Inc,"[Apple Inc, Nike Inc, Apple Retail Uk Ltd]",AAPL.O
2,Andrew R. Jassy,"President, Chief Executive Officer, Director",Amazon.com Inc,"[Coupa Software Inc, Amazon.com Inc]",AMZN.O
3,Sundar Pichai,"Chief Executive Officer, Director",Alphabet Inc,"[Alphabet Inc, Ruba Inc (Fka Kudo Inc)]",GOOGL.O
4,Mark Zuckerberg,"Chairman of the Board, Chief Executive Officer, Founder",Meta Platforms Inc,"[Chan Zuckerberg Initiative LLC, Meta Platforms Inc]",FB.O
5,Warren E. Buffett,"Chairman of the Board, Chief Executive Officer",Berkshire Hathaway Inc,"[Grinnell College, Bill & Melinda Gates Foundation Trust, Berkshire Hathaway Inc, Kraft Heinz Intermediate Corporation II, Kraft Heinz F...",BRKa
6,Daniel Zhang,"Chairman of the Board, Chief Executive Officer",Alibaba Group Holding Ltd,"[Alibaba Group Holding Ltd, DiDi Global Inc]",BABA.K
7,Fan Jiang,President - Taobao and Tmall,Alibaba Group Holding Ltd,"[Alibaba Group Holding Ltd, Umeng Technology Ltd]",BABA.K
8,Huateng Ma,"Executive Chairman of the Board, Chief Executive Officer",Tencent Holdings Ltd,"[Tencent Holdings Ltd, Tencent Technology Beijing Co Ltd]",0700.HK
9,Al Francis Kelly,"Chairman of the Board, Chief Executive Officer",Visa Inc,[Visa Inc],V


#### Example - Live Options
The following example will specify an underlying instrument (RIC) and pull the list of options that are active, i.e. not expired.

In [20]:
# Define our underlying
underlying = "'AAPL.O'"

# To search for our instrument, I'm utilizing the property: 'UnderlyingQuoteRIC'.  The property does not support an 
# exact match 'xeq', so we must use the partial match 'eq' to find all hits. Depending on the instrument you specify, 
# this may introduce additional hits.  For example, if you are using the underlying 'AAPL.O', search will also find a 
# hit on 'AAPL.OQ', for example. To protect against this, we can utilize a combination of the built-in functions, 
# 'startswith' and 'endwith' to perform our match.
response=search.Definition(
    view = search.SearchViews.EQUITY_DERIVATIVE_QUOTES,
    filter = "AssetState eq 'AC' and \
              startswith(UnderlyingQuoteRIC, " + underlying + ") and \
              endswith(UnderlyingQuoteRIC," + underlying + ") and \
              RCSAssetClass eq 'OPT' and IsChain eq false and \
              ExpiryDate ge " + dt.datetime.today().strftime('%Y-%m-%d'),
    select = "DocumentTitle, RIC, ExpiryDate, CallPutOption, StrikePrice",
    top = 10000
).get_data()
response.data.df

,DocumentTitle,RIC,ExpiryDate,CallPutOption,StrikePrice
0,"OPRA Apple Equity Option 150 Put Apr 2022 , Equity Cash Option, Put 150 USD 14-Apr-2022, OPRA",AAPLP142215000.U,2022-04-14T00:00:00.000Z,Put,150.0
1,"OPRA Apple Equity Option 155 Put Apr 2022 , Equity Cash Option, Put 155 USD 14-Apr-2022, OPRA",AAPLP142215500.U,2022-04-14T00:00:00.000Z,Put,155.0
2,"OPRA Apple Equity Option 160 Put Apr 2022 , Equity Cash Option, Put 160 USD 14-Apr-2022, OPRA",AAPLP142216000.U,2022-04-14T00:00:00.000Z,Put,160.0
3,"OPRA Apple Equity Option 165 Call Apr 2022 , Equity Cash Option, Call 165 USD 14-Apr-2022, OPRA",AAPLD142216500.U,2022-04-14T00:00:00.000Z,Call,165.0
4,"OPRA Apple Equity Option 160 Put Jun 2022 , Equity Cash Option, Put 160 USD 17-Jun-2022, OPRA",AAPLR172216000.U,2022-06-17T00:00:00.000Z,Put,160.0
...,...,...,...,...,...
1811,"CBOE Apple PM Settled European Flex Equity option 182 Call Jun 2022 , Equity Cash Option, Call 182 USD 30-Jun-2022, Chicago Board Option...",2AAPLF3022D00182000.FO,2022-06-30T00:00:00.000Z,Call,182.0
1812,"CBOE Apple PM Settled European Flex Equity option 187.5 Call Jun 2022 , Equity Cash Option, Call 187.5 USD 03-Jun-2022, Chicago Board Op...",2AAPLF0322D00187500.FO,2022-06-03T00:00:00.000Z,Call,187.5
1813,"CBOE Apple PM Settled European Flex Equity option 182.5 Call May 2022 , Equity Cash Option, Call 182.5 USD 06-May-2022, Chicago Board Op...",2AAPLE0622D00182500.FO,2022-05-06T00:00:00.000Z,Call,182.5
1814,"CBOE Apple PM Settled European Flex Equity option 156.01 Call Mar 2022 , Equity Cash Option, Call 156.01 USD 24-Mar-2022, Chicago Board ...",2AAPLC2422D00156010.FO,2022-03-24T00:00:00.000Z,Call,156.01
